In [1]:
import os
import sys
import time
import math
import tarfile
import zipfile
import PIL.Image
import numpy as np
import tensorflow as tf
import six.moves.urllib as urllib
# from io import StringIO
# from  cStringIO import StringIO
from io import BytesIO as StringIO
from IPython.display import clear_output, Image, display, HTML
from object_detection.utils import label_map_util
from object_detection.utils.visualization_utils import visualize_boxes_and_labels_on_image_array
from object_detection.core import target_assigner

In [2]:
from .. import utils
import pandas as pd

# Your variables

Path for images (for inference)

In [ ]:
YOUR_PATH = "This/is/my/path/to/images"

Choose the target class for perturbation

In [ ]:
TARGET_CLASS = 3 # target class: person 1 car 3 

#Model preparation

In [3]:
OBJ_DETECT_API='~/data/'

# MODEL_NAME = 'faster_rcnn_inception_v2_coco_2017_11_08'
MODEL_NAME = 'ssd_inception_v2_coco_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = './data/mscoco_label_map.pbtxt'

NUM_CLASSES = 90

# Download Model

We download the model from Tensorflow object detection lib : 'http://download.tensorflow.org/models/object_detection/'

In [4]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename("./"+file.name)
  print(file_name)
  if 'frozen_inference_graph.pb' in file_name:
    print(file, os.getcwd())
    tar_file.extract(file, os.getcwd())

ssd_inception_v2_coco_2018_01_28
model.ckpt.index
checkpoint
pipeline.config
model.ckpt.data-00000-of-00001
model.ckpt.meta
saved_model
saved_model.pb
variables
frozen_inference_graph.pb
<TarInfo 'ssd_inception_v2_coco_2018_01_28/frozen_inference_graph.pb' at 0x7f3eed5bc110> /usr/users/attaquesetdefensessurlia/godetrem


# Loading label map

In [5]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Utility functions for loading and displaying images

In [6]:
def read_image(path):
    img = PIL.Image.open(path)
    img = np.array(img, dtype=np.uint8)
    return img

def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))



# tests 
img = read_image('data/img_stop_sign.png')
img3 = utils.read_image_resize('data/1.JPG', crop=True)

# Loading the mask
There are 2 masks, one for the red part of stop sign and one for the white part.

In [7]:
# size of the perturbation we want to generate
psize = 300
WHITE_MASK = np.array(PIL.Image.fromarray(np.load('data/stop_white_mask.npy')).resize((psize, psize)))
RED_MASK = np.array(PIL.Image.fromarray(np.load('data/stop_red_mask.npy')).resize((psize, psize))) # the one we modify

# Loading the model for inference

In [10]:
inference_graph = tf.Graph()
with inference_graph.as_default():
    image_tensor = tf.placeholder(tf.float32, shape=(None, psize, psize, 3), name='image_tensor')
    inference_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        inference_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(inference_graph_def, name='',
                            input_map={'Preprocessor/map/TensorArrayStack/TensorArrayGatherV3:0':image_tensor})

# Defining some plotting utility functions

In [11]:
def plot_detections(img, scores=None, bboxes=None, min_threshold=0):
    if scores is None or bboxes is None:
        
        inference_sess = tf.Session(graph=inference_graph)
        # print(inference_sess.graph_def)
        tensors = [ inference_graph.get_tensor_by_name('detection_boxes:0'),
                    inference_graph.get_tensor_by_name('detection_scores:0'),
                    inference_graph.get_tensor_by_name('detection_classes:0'),
                    inference_graph.get_tensor_by_name('num_detections:0'),
                    # inference_graph.get_tensor_by_name('Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Reshape_4:0'),
                    inference_graph.get_tensor_by_name('Postprocessor/convert_scores:0') ]

        feed_dict = { inference_graph.get_tensor_by_name('image_tensor:0'): np.expand_dims(img, axis=0),inference_graph.get_tensor_by_name('image_tensor_1:0'): np.expand_dims(img, axis=0) }

        nms_bboxes, nms_scores, nms_classes, num_detections, scores = inference_sess.run(tensors,
                                                                                                 feed_dict)
        
        # bboxes = bboxes[0]
        scores = scores[0]
    
    sorted_classes = np.argsort(scores[:, 1:], axis=1)
    sorted_scores = scores[:, 1:].copy()
    # sorted_bboxes = bboxes.copy()

    for i, ordering in enumerate(sorted_classes):
        sorted_scores[i, :] = scores[i, ordering+1]
        # sorted_bboxes[i, :] = bboxes[i, ordering, :]

    sorted_classes += 1
    return sorted_scores, sorted_classes


In [12]:
# Test detections on a given image from dataset
path = f"{YOUR_PATH}/shapeshifter_car_3.4°_0°_5m_f18mm_a0_185lux.JPG"
plot_detections(utils.read_image_resize(path,height=psize, width=psize, zoom=1.2), min_threshold=0.5)

(array([[1.6778708e-05, 1.7732382e-05, 1.9699335e-05, ..., 2.8591156e-03,
         2.9899180e-03, 6.3054860e-03],
        [4.6044588e-05, 5.3942204e-05, 5.8412552e-05, ..., 1.8167198e-03,
         2.0538867e-03, 2.2564232e-03],
        [3.5256147e-05, 5.0067902e-05, 5.1081181e-05, ..., 9.8779798e-04,
         1.9553900e-03, 3.9180517e-03],
        ...,
        [5.8114529e-05, 9.0032816e-05, 9.8526478e-05, ..., 1.1786819e-03,
         1.2669265e-03, 1.7919540e-03],
        [3.2067299e-05, 4.7683716e-05, 5.8978796e-05, ..., 8.0490112e-04,
         1.5103817e-03, 2.1658242e-03],
        [3.6358833e-06, 3.7252903e-06, 6.0498714e-06, ..., 1.4872253e-03,
         5.5933297e-03, 1.3913006e-02]], dtype=float32),
 array([[34, 40, 23, ..., 62, 84,  1],
        [16, 34, 38, ..., 78,  1, 62],
        [40, 35, 34, ..., 79, 62,  1],
        ...,
        [18,  4, 52, ...,  7,  5, 78],
        [18,  4, 88, ...,  9,  1, 82],
        [ 4, 40, 18, ...,  7,  1, 82]]))

# Compute model performance on images

In [13]:

def get_img_detection_data_from_path(img_path, image_folder_path, image_focal=None, zoom=None, scores=None, bboxes=None, min_threshold=0):
    img = utils.get_img_cropped(image_folder_path, img_path, f=image_focal, max_crop = 0.4, min_crop=1, max_f=250, min_f=55, zoom=zoom, height=300, width=300)
    if scores is None or bboxes is None:
        
        inference_sess = tf.Session(graph=inference_graph)
        # print(inference_sess.graph_def)
        tensors = [ inference_graph.get_tensor_by_name('detection_boxes:0'),
                    inference_graph.get_tensor_by_name('detection_scores:0'),
                    inference_graph.get_tensor_by_name('detection_classes:0'),
                    inference_graph.get_tensor_by_name('num_detections:0'),
                    # inference_graph.get_tensor_by_name('Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/Reshape_4:0'),
                    inference_graph.get_tensor_by_name('Postprocessor/convert_scores:0') ]

        feed_dict = { inference_graph.get_tensor_by_name('image_tensor:0'): np.expand_dims(img, axis=0),inference_graph.get_tensor_by_name('image_tensor_1:0'): np.expand_dims(img, axis=0) }

        nms_bboxes, nms_scores, nms_classes, num_detections, scores = inference_sess.run(tensors,
                                                                                                 feed_dict)
        
        # bboxes = bboxes[0]
        scores = scores[0]
    
    sorted_classes = np.argsort(scores[:, 1:], axis=1)
    sorted_scores = scores[:, 1:].copy()
    # sorted_bboxes = bboxes.copy()

    for i, ordering in enumerate(sorted_classes):
        sorted_scores[i, :] = scores[i, ordering+1]
        # sorted_bboxes[i, :] = bboxes[i, ordering, :]

    sorted_classes += 1
    # return sorted_scores, sorted_classes
    return sorted_classes, sorted_scores

Automatic zoom ratio

In [14]:
# get images inferences - approx 2 minutes
image_folder_path = YOUR_PATH
# load images names
images_autocrop_df = utils.load_images_df(image_folder_path)
# infer
images_autocrop_df[["sorted_classes" , "sorted_scores"]] = images_autocrop_df[["filename", "Focale"]].apply(lambda x:pd.Series(get_img_detection_data_from_path(x[0], image_folder_path, image_focal=x[1])), axis=1)

With a fixed zoom ratio

In [ ]:
# get images inferences - approx 15 minutes
images_dfs_zoom = {}
image_folder_path = YOUR_PATH
for zoom in [1,0.6,0.4]: # chosen fixed zooms 
    # load images 
    df = utils.load_images_df(image_folder_path)
    # infer on a column
    df[["sorted_classes" , "sorted_scores", "sorted_bboxes"]] = df[["filename", "Focale"]].apply(lambda x:pd.Series(get_img_detection_data_from_path(x[0], image_folder_path, zoom=zoom)), axis=1)
    images_dfs_zoom[zoom] = df

In [ ]:
def accuracy(target_class,sorted_classes, sorted_scores,treshold=0.5, debug=None):
    '''
    The accuracy function : computes (in a range of classes, and over a treshold) the number of 
    correctly recognized classes over the total number of classes. 
    '''
    if debug is not None:
        print("=======================")
        print(debug)
    temp = 0
    targets = 0
    # print(target_class)
    # print(sorted_classes)
    # print(sorted_scores)
    for i in range(len(sorted_classes)):
        for j in range(len(sorted_classes[i])):
            if sorted_classes[i][j] in [13, 1 , 3]:
                if sorted_scores[i][j]>=treshold:
                    temp += 1
                    print(sorted_classes[i][j], sorted_scores[i][j])
                    if sorted_classes[i][j]==target_class:
                        targets += 1
    if temp==0:
        return None
    # print(targets, temp)
    return targets  / temp      



In [ ]:
# Compute accuracy in forced zooms 
for zoom in images_dfs_zoom.keys():
    image_df = images_dfs_zoom[zoom]
    image_df["AccuracyPerson"] = image_df.apply(lambda x: accuracy(1, x.sorted_classes, x.sorted_scores), axis=1)
    image_df["AccuracyCar"] = image_df.apply(lambda x: accuracy(3, x.sorted_classes, x.sorted_scores), axis=1)
    image_df["AccuracyStopSign"] = image_df.apply(lambda x: accuracy(13, x.sorted_classes, x.sorted_scores), axis=1)
# Compute accuracy in autozoom
images_autocrop_df["AccuracyPerson"] = images_autocrop_df.apply(lambda x: accuracy(1, x.sorted_classes, x.sorted_scores), axis=1)
images_autocrop_df["AccuracyCar"] = images_autocrop_df.apply(lambda x: accuracy(3, x.sorted_classes, x.sorted_scores), axis=1)
images_autocrop_df["AccuracyStopSign"] = images_autocrop_df.apply(lambda x: accuracy(13, x.sorted_classes, x.sorted_scores), axis=1)

In [ ]:
# insights
# control_df[["filename", "Accuracy"]]
# person_df[["filename", "Accuracy"]][person_df["Accuracy"]<=1]
# car_df[["filename", "sorted_classes", "Accuracy"]]


In [ ]:
# extract values as list (pandas fun)
images_autocrop_df["AccuracyPersonForcedZoom"] = images_autocrop_df.apply(lambda x: [df[df["filename"]==x[0]]["AccuracyPerson"].iloc[0] 
                                                                               for df in images_dfs_zoom.values()], axis=1)
images_autocrop_df["AccuracyCarForcedZoom"] = images_autocrop_df.apply(lambda x: [df[df["filename"]==x[0]]["AccuracyCar"].iloc[0] 
                                                                               for df in images_dfs_zoom.values()], axis=1)
images_autocrop_df["AccuracyStopSignForcedZoom"] = images_autocrop_df.apply(lambda x: [df[df["filename"]==x[0]]["AccuracyStopSign"].iloc[0] 
                                                                               for df in images_dfs_zoom.values()], axis=1)

We can save as pickle (but this is sensible to modules version)

In [ ]:
# images_autocrop_df.to_pickle("dataframe_ssd")
# df = pd.read_pickle("dataframe")
# df

It is better to save as Excel: we can load it in windows, or use a lighter environment to do data analysis, see visualisation.py for this step

In [ ]:
df.to_excel("results.xlsx")

# Create the images 

This is directly from the article's code

Define the optimization model

In [ ]:
class ModelContainer():
    def __init__(self):
        self.graph = tf.Graph()
        self.sess = tf.Session(graph=self.graph)
        self.patch_shape = (psize, psize, 3)
        self.batch_size_ = 10
        self._make_model_and_ops(None)

    def get_patch(self):
        patch = np.round((self._run(self.clipped_patch_)+1)*(255/2.0)).astype(np.uint8)
        patch *= RED_MASK
        patch[patch == 0] = 255
        return patch

    def assign_patch(self, new_patch):
        self._run(self.assign_patch_, {self.patch_placeholder_: new_patch})

    def reset_patch(self):
        self.assign_patch(np.zeros(self.patch_shape))
          
    def train_step(self, images, patch_transforms, second_stage_cls_labels, learning_rate=1.0,
                   dropout=None, rpn_nms_bboxes=None, rpn_nms_indices=None, patch_loss_weight=None):
        if (rpn_nms_bboxes is None) or \
           (rpn_nms_indices is None):
            rpn_nms_bboxes, rpn_nms_indices = self.inference_rpn(images, patch_transforms)

        feed_dict = { self.image_input_: images,
                      self.patch_transforms_: patch_transforms,
                      self.second_stage_cls_labels_: second_stage_cls_labels,
                      self.rpn_nms_bboxes_placeholder_: rpn_nms_bboxes,
                      self.rpn_nms_indices_placeholder_: rpn_nms_indices,
                      self.learning_rate_: learning_rate }
        
        if patch_loss_weight is not None:
            feed_dict[self.patch_loss_weight_] = patch_loss_weight
        
        tensors = [ self.train_op_,
                    self.loss_,
                    self.second_stage_cls_loss_, 
                    self.patch_loss_]

        train_op, loss, second_stage_cls_loss, patch_loss = self._run(tensors, feed_dict, dropout=dropout)

        return loss, second_stage_cls_loss, patch_loss
    
    def inference_rpn(self, images, patch_transforms):
        feed_dict = { self.image_input_: images,
                      self.patch_transforms_: patch_transforms }
        
        tensors = [self.rpn_nms_bboxes_,
                   self.rpn_nms_indices_ ]

        rpn_nms_bboxes, rpn_nms_indices = self._run(tensors, feed_dict)
        
        return rpn_nms_bboxes, rpn_nms_indices

    def inference(self, images, patch_transforms, rpn_nms_bboxes=None, rpn_nms_indices=None):
        if (rpn_nms_bboxes is None) or \
           (rpn_nms_indices is None):
            rpn_nms_bboxes, rpn_nms_indices = self.inference_rpn(images, patch_transforms)

        feed_dict = { self.image_input_: images,
                      self.patch_transforms_: patch_transforms,
                      self.rpn_nms_bboxes_placeholder_: rpn_nms_bboxes,
                      self.rpn_nms_indices_placeholder_: rpn_nms_indices }

        tensors = [ self.patched_input_,
                    self.second_stage_cls_scores_,
                    self.second_stage_loc_bboxes_ ]

        patched_imgs, second_stage_cls_scores, second_stage_loc_bboxes = self._run(tensors, feed_dict)
        patched_imgs = patched_imgs.astype(np.uint8)

        plot_detections(patched_imgs[0], scores=second_stage_cls_scores[0], bboxes=second_stage_loc_bboxes[0], min_threshold=0.2)
        
        return patched_imgs, second_stage_cls_scores, second_stage_loc_bboxes

    def _run(self, target, feed_dict=None, dropout=None):
        if feed_dict is None:
            feed_dict = {}
        
        if dropout is not None:
            feed_dict[self.dropout_] = dropout
    
        return self.sess.run(target, feed_dict=feed_dict)
    
    def _make_model_and_ops(self, patch_val):
        start = time.time()
        with self.sess.graph.as_default():
            tf.set_random_seed(1234)
            
            # Tensors are post-fixed with an underscore!
            self.image_input_ = tf.placeholder(tf.float32, shape=(None, psize, psize, 3), name='image_input')
            self.patch_transforms_ = tf.placeholder(tf.float32, shape=(None, 8), name='patch_transforms')

            patch_ = tf.get_variable('patch', self.patch_shape, dtype=tf.float32, initializer=tf.zeros_initializer)
            self.patch_placeholder_ = tf.placeholder(dtype=tf.float32, shape=self.patch_shape, name='patch_placeholder')
            self.assign_patch_ = tf.assign(patch_, self.patch_placeholder_)
            self.clipped_patch_ = tf.tanh(patch_)

            self.dropout_ = tf.placeholder_with_default(1.0, [], name='dropout')
            patch_with_dropout_ = tf.nn.dropout(self.clipped_patch_, keep_prob=self.dropout_)
            patched_input_ = tf.clip_by_value(self._random_overlay(self.image_input_, patch_with_dropout_), clip_value_min=-1.0, clip_value_max=1.0)
            patched_input_ = tf.clip_by_value(tf.image.random_brightness(patched_input_, 10.0/255), -1.0, 1.0)
            self.patched_input_ = tf.fake_quant_with_min_max_vars((patched_input_ + 1)*127.5, min=0, max=255)

            # Create placeholders for NMS RPN inputs
            self.rpn_nms_bboxes_placeholder_ = tf.placeholder(tf.float32, shape=(None, 4), name='rpn_nms_bboxes')
            self.rpn_nms_indices_placeholder_ = tf.placeholder(tf.int32, shape=(None), name='rpn_nms_indices')

            detection_graph_def = tf.GraphDef()
            with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
                serialized_graph = fid.read()
                detection_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(detection_graph_def, name='detection',
                                    input_map={
                                               'Preprocessor/map/TensorArrayStack/TensorArrayGatherV3:0':self.patched_input_,
                                               'Reshape_7:0':self.rpn_nms_bboxes_placeholder_,
                                               'Reshape_8:0':self.rpn_nms_indices_placeholder_,
                                              })

            # Recreate tensors we just replaced in the input_map
            self.rpn_nms_bboxes_ = tf.reshape(self.graph.get_tensor_by_name('detection/Reshape_6:0'), self.graph.get_tensor_by_name('detection/stack_3:0'), name='detection/Reshape_7')
            self.rpn_nms_indices_ = tf.reshape(self.graph.get_tensor_by_name('detection/mul_1:0'), self.graph.get_tensor_by_name('detection/Reshape_8/shape:0'), name='detection/Reshape_8') 

            # Patch Loss
            self.patch_loss_ = tf.nn.l2_loss(RED_MASK*(self.clipped_patch_ - np.tile(np.array([ 1.0, -0.9, -1]), (psize, psize, 1))))
            self.patch_loss_weight_ = tf.placeholder_with_default(1.0, [], 'patch_loss_weight')

            # Second-stage Class Loss
            self.second_stage_cls_scores_ = self.graph.get_tensor_by_name('detection/SecondStagePostprocessor/convert_scores:0')
            second_stage_cls_logits_ = self.graph.get_tensor_by_name('detection/SecondStagePostprocessor/scale_logits:0')
            self.second_stage_cls_labels_ = tf.placeholder(tf.float32, shape=second_stage_cls_logits_.shape, name='second_stage_cls_labels')
            second_stage_cls_losses_ = tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.reshape(self.second_stage_cls_labels_, (-1, self.second_stage_cls_labels_.shape[2])),
                                                                                      logits=tf.reshape(second_stage_cls_logits_, (-1, second_stage_cls_logits_.shape[2]))) 
            second_stage_cls_losses_ = tf.reshape(second_stage_cls_losses_, (-1, self.second_stage_cls_labels_.shape[1]))
            second_stage_cls_losses_ = tf.divide(second_stage_cls_losses_, tf.to_float(self.second_stage_cls_labels_.shape[1]))
            self.second_stage_cls_loss_ = tf.reduce_sum(second_stage_cls_losses_)
           
            # Second-stage bounding boxes
            self.second_stage_loc_bboxes_ = self.graph.get_tensor_by_name('detection/SecondStagePostprocessor/Reshape_4:0')
    
            # Sum of weighted losses
            self.loss_ = self.patch_loss_*self.patch_loss_weight_ + (self.second_stage_cls_loss_)

            # Train our attack by only training on the patch variable
            self.learning_rate_ = tf.placeholder(tf.float32)
            self.train_op_ = tf.train.GradientDescentOptimizer(self.learning_rate_).minimize(self.loss_, var_list=[patch_])
            
            if patch_val is not None:
                self.assign_patch(patch_val)
            else:
                self.reset_patch()

            elapsed = time.time() - start
            print("Finished loading the model, took {:.0f}s".format(elapsed))
    

    def _random_overlay(self, imgs, patch):    
        red_mask = RED_MASK.astype(np.float32)
        white_mask = WHITE_MASK.astype(np.float32)
        
        red_mask = tf.stack([red_mask] * self.batch_size_)
        white_mask = tf.stack([white_mask] * self.batch_size_)
        padded_patch = tf.stack([patch] * self.batch_size_)
        
        white = tf.ones_like(red_mask) * 0.95
              
        red_mask = tf.contrib.image.transform(red_mask, self.patch_transforms_, 'BILINEAR')
        white_mask = tf.contrib.image.transform(white_mask, self.patch_transforms_, 'BILINEAR')
        padded_patch = tf.contrib.image.transform(padded_patch, self.patch_transforms_, 'BILINEAR')

        inverted_mask = (1 - red_mask - white_mask)

        return white * white_mask + imgs * inverted_mask + padded_patch * red_mask
    

    def _transform_vector(self, width, x_shift, y_shift, im_scale, rot_in_degrees):
        """
        If one row of transforms is [a0, a1, a2, b0, b1, b2, c0, c1], 
        then it maps the output point (x, y) to a transformed input point 
        (x', y') = ((a0 x + a1 y + a2) / k, (b0 x + b1 y + b2) / k), 
        where k = c0 x + c1 y + 1. 
        The transforms are inverted compared to the transform mapping input points to output points.
        """

        rot = float(rot_in_degrees) / 90. * (math.pi/2)

        # Standard rotation matrix
        # (use negative rot because tf.contrib.image.transform will do the inverse)
        rot_matrix = np.array(
            [[math.cos(-rot), -math.sin(-rot)],
            [math.sin(-rot), math.cos(-rot)]]
        )

        # Scale it
        # (use inverse scale because tf.contrib.image.transform will do the inverse)
        inv_scale = 1. / im_scale 
        xform_matrix = rot_matrix * inv_scale
        a0, a1 = xform_matrix[0]
        b0, b1 = xform_matrix[1]

        # At this point, the image will have been rotated around the top left corner,
        # rather than around the center of the image. 
        #
        # To fix this, we will see where the center of the image got sent by our transform,
        # and then undo that as part of the translation we apply.
        x_origin = float(width) / 2
        y_origin = float(width) / 2

        x_origin_shifted, y_origin_shifted = np.matmul(
            xform_matrix,
            np.array([x_origin, y_origin]),
        )

        x_origin_delta = x_origin - x_origin_shifted
        y_origin_delta = y_origin - y_origin_shifted

        # Combine our desired shifts with the rotation-induced undesirable shift
        a2 = x_origin_delta - (x_shift/(2*im_scale))
        b2 = y_origin_delta - (y_shift/(2*im_scale))

        # Return these values in the order that tf.contrib.image.transform expects
        return np.array([a0, a1, a2, b0, b1, b2, 0, 0]).astype(np.float32)

    def generate_random_transformation(self, scale_min=0.2, scale_max=0.6, width=psize, max_rotation=20):
        im_scale = np.random.uniform(low=scale_min, high=scale_max)

        padding_after_scaling = (1-im_scale) * width
        x_delta = np.random.uniform(-padding_after_scaling, padding_after_scaling)
        y_delta = np.random.uniform(-padding_after_scaling, padding_after_scaling)

        rot = np.random.uniform(-max_rotation, max_rotation)

        return self._transform_vector(width, 
                                      x_shift=x_delta,
                                      y_shift=y_delta,
                                      im_scale=im_scale, 
                                      rot_in_degrees=rot)    

model = ModelContainer()

In [ ]:
class ModelContainer():
    def __init__(self):
        self.graph = tf.Graph()
        self.sess = tf.Session(graph=self.graph)
        self.patch_shape = (psize, psize, 3)
        self.batch_size_ = 10
        self._make_model_and_ops(None)

    def get_patch(self):
        patch = np.round((self._run(self.clipped_patch_)+1)*(255/2.0)).astype(np.uint8)
        patch *= RED_MASK
        patch[patch == 0] = 255
        return patch

    def assign_patch(self, new_patch):
        self._run(self.assign_patch_, {self.patch_placeholder_: new_patch})

    def reset_patch(self):
        self.assign_patch(np.zeros(self.patch_shape))
          
    def train_step(self, images, patch_transforms, second_stage_cls_labels, learning_rate=1.0,
                   dropout=None, rpn_nms_bboxes=None, rpn_nms_indices=None, patch_loss_weight=None):
        if (rpn_nms_bboxes is None) or \
           (rpn_nms_indices is None):
            rpn_nms_bboxes, rpn_nms_indices = self.inference_rpn(images, patch_transforms)

        feed_dict = { self.image_input_: images,
                      self.patch_transforms_: patch_transforms,
                      self.second_stage_cls_labels_: second_stage_cls_labels,
                      self.rpn_nms_bboxes_placeholder_: rpn_nms_bboxes,
                      self.rpn_nms_indices_placeholder_: rpn_nms_indices,
                      self.learning_rate_: learning_rate }
        
        if patch_loss_weight is not None:
            feed_dict[self.patch_loss_weight_] = patch_loss_weight
        
        tensors = [ self.train_op_,
                    self.loss_,
                    self.second_stage_cls_loss_, 
                    self.patch_loss_]

        train_op, loss, second_stage_cls_loss, patch_loss = self._run(tensors, feed_dict, dropout=dropout)

        return loss, second_stage_cls_loss, patch_loss
    
    def inference_rpn(self, images, patch_transforms):
        feed_dict = { self.image_input_: images,
                      self.patch_transforms_: patch_transforms }
        
        tensors = [self.rpn_nms_bboxes_,
                   self.rpn_nms_indices_ ]

        rpn_nms_bboxes, rpn_nms_indices = self._run(tensors, feed_dict)
        
        return rpn_nms_bboxes, rpn_nms_indices

    def inference(self, images, patch_transforms, rpn_nms_bboxes=None, rpn_nms_indices=None):
        if (rpn_nms_bboxes is None) or \
           (rpn_nms_indices is None):
            rpn_nms_bboxes, rpn_nms_indices = self.inference_rpn(images, patch_transforms)

        feed_dict = { self.image_input_: images,
                      self.patch_transforms_: patch_transforms,
                      self.rpn_nms_bboxes_placeholder_: rpn_nms_bboxes,
                      self.rpn_nms_indices_placeholder_: rpn_nms_indices }

        tensors = [ self.patched_input_,
                    self.second_stage_cls_scores_,
                    self.second_stage_loc_bboxes_ ]

        patched_imgs, second_stage_cls_scores, second_stage_loc_bboxes = self._run(tensors, feed_dict)
        patched_imgs = patched_imgs.astype(np.uint8)

        plot_detections(patched_imgs[0], scores=second_stage_cls_scores[0], bboxes=second_stage_loc_bboxes[0], min_threshold=0.2)
        
        return patched_imgs, second_stage_cls_scores, second_stage_loc_bboxes

    def _run(self, target, feed_dict=None, dropout=None):
        if feed_dict is None:
            feed_dict = {}
        
        if dropout is not None:
            feed_dict[self.dropout_] = dropout
    
        return self.sess.run(target, feed_dict=feed_dict)
    
    def _make_model_and_ops(self, patch_val):
        start = time.time()
        with self.sess.graph.as_default():
            tf.set_random_seed(1234)
            
            # Tensors are post-fixed with an underscore!
            self.image_input_ = tf.placeholder(tf.float32, shape=(None, psize, psize, 3), name='image_input')
            self.patch_transforms_ = tf.placeholder(tf.float32, shape=(None, 8), name='patch_transforms')

            patch_ = tf.get_variable('patch', self.patch_shape, dtype=tf.float32, initializer=tf.zeros_initializer)
            self.patch_placeholder_ = tf.placeholder(dtype=tf.float32, shape=self.patch_shape, name='patch_placeholder')
            self.assign_patch_ = tf.assign(patch_, self.patch_placeholder_)
            self.clipped_patch_ = tf.tanh(patch_)

            self.dropout_ = tf.placeholder_with_default(1.0, [], name='dropout')
            patch_with_dropout_ = tf.nn.dropout(self.clipped_patch_, keep_prob=self.dropout_)
            patched_input_ = tf.clip_by_value(self._random_overlay(self.image_input_, patch_with_dropout_), clip_value_min=-1.0, clip_value_max=1.0)
            patched_input_ = tf.clip_by_value(tf.image.random_brightness(patched_input_, 10.0/255), -1.0, 1.0)
            self.patched_input_ = tf.fake_quant_with_min_max_vars((patched_input_ + 1)*127.5, min=0, max=255)

            # Create placeholders for NMS RPN inputs
            self.rpn_nms_bboxes_placeholder_ = tf.placeholder(tf.float32, shape=(None, 4), name='rpn_nms_bboxes')
            self.rpn_nms_indices_placeholder_ = tf.placeholder(tf.int32, shape=(None), name='rpn_nms_indices')

            detection_graph_def = tf.GraphDef()
            with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
                serialized_graph = fid.read()
                detection_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(detection_graph_def, name='detection',
                                    input_map={
                                               'Preprocessor/map/TensorArrayStack/TensorArrayGatherV3:0':self.patched_input_,
                                               'Reshape_7:0':self.rpn_nms_bboxes_placeholder_,
                                               'Reshape_8:0':self.rpn_nms_indices_placeholder_,
                                              })

            # Recreate tensors we just replaced in the input_map
            self.rpn_nms_bboxes_ = tf.reshape(self.graph.get_tensor_by_name('detection/Reshape_6:0'), self.graph.get_tensor_by_name('detection/stack_3:0'), name='detection/Reshape_7')
            self.rpn_nms_indices_ = tf.reshape(self.graph.get_tensor_by_name('detection/mul_1:0'), self.graph.get_tensor_by_name('detection/Reshape_8/shape:0'), name='detection/Reshape_8') 

            # Patch Loss
            self.patch_loss_ = tf.nn.l2_loss(RED_MASK*(self.clipped_patch_ - np.tile(np.array([ 1.0, -0.9, -1]), (psize, psize, 1))))
            self.patch_loss_weight_ = tf.placeholder_with_default(1.0, [], 'patch_loss_weight')

            # Second-stage Class Loss
            self.second_stage_cls_scores_ = self.graph.get_tensor_by_name('detection/SecondStagePostprocessor/convert_scores:0')
            second_stage_cls_logits_ = self.graph.get_tensor_by_name('detection/SecondStagePostprocessor/scale_logits:0')
            self.second_stage_cls_labels_ = tf.placeholder(tf.float32, shape=second_stage_cls_logits_.shape, name='second_stage_cls_labels')
            second_stage_cls_losses_ = tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.reshape(self.second_stage_cls_labels_, (-1, self.second_stage_cls_labels_.shape[2])),
                                                                                      logits=tf.reshape(second_stage_cls_logits_, (-1, second_stage_cls_logits_.shape[2]))) 
            second_stage_cls_losses_ = tf.reshape(second_stage_cls_losses_, (-1, self.second_stage_cls_labels_.shape[1]))
            second_stage_cls_losses_ = tf.divide(second_stage_cls_losses_, tf.to_float(self.second_stage_cls_labels_.shape[1]))
            self.second_stage_cls_loss_ = tf.reduce_sum(second_stage_cls_losses_)
           
            # Second-stage bounding boxes
            self.second_stage_loc_bboxes_ = self.graph.get_tensor_by_name('detection/SecondStagePostprocessor/Reshape_4:0')
    
            # Sum of weighted losses
            self.loss_ = self.patch_loss_*self.patch_loss_weight_ + (self.second_stage_cls_loss_)

            # Train our attack by only training on the patch variable
            self.learning_rate_ = tf.placeholder(tf.float32)
            self.train_op_ = tf.train.GradientDescentOptimizer(self.learning_rate_).minimize(self.loss_, var_list=[patch_])
            
            if patch_val is not None:
                self.assign_patch(patch_val)
            else:
                self.reset_patch()

            elapsed = time.time() - start
            print("Finished loading the model, took {:.0f}s".format(elapsed))
    

    def _random_overlay(self, imgs, patch):    
        red_mask = RED_MASK.astype(np.float32)
        white_mask = WHITE_MASK.astype(np.float32)
        
        red_mask = tf.stack([red_mask] * self.batch_size_)
        white_mask = tf.stack([white_mask] * self.batch_size_)
        padded_patch = tf.stack([patch] * self.batch_size_)
        
        white = tf.ones_like(red_mask) * 0.95
              
        red_mask = tf.contrib.image.transform(red_mask, self.patch_transforms_, 'BILINEAR')
        white_mask = tf.contrib.image.transform(white_mask, self.patch_transforms_, 'BILINEAR')
        padded_patch = tf.contrib.image.transform(padded_patch, self.patch_transforms_, 'BILINEAR')

        inverted_mask = (1 - red_mask - white_mask)

        return white * white_mask + imgs * inverted_mask + padded_patch * red_mask
    

    def _transform_vector(self, width, x_shift, y_shift, im_scale, rot_in_degrees):
        """
        If one row of transforms is [a0, a1, a2, b0, b1, b2, c0, c1], 
        then it maps the output point (x, y) to a transformed input point 
        (x', y') = ((a0 x + a1 y + a2) / k, (b0 x + b1 y + b2) / k), 
        where k = c0 x + c1 y + 1. 
        The transforms are inverted compared to the transform mapping input points to output points.
        """

        rot = float(rot_in_degrees) / 90. * (math.pi/2)

        # Standard rotation matrix
        # (use negative rot because tf.contrib.image.transform will do the inverse)
        rot_matrix = np.array(
            [[math.cos(-rot), -math.sin(-rot)],
            [math.sin(-rot), math.cos(-rot)]]
        )

        # Scale it
        # (use inverse scale because tf.contrib.image.transform will do the inverse)
        inv_scale = 1. / im_scale 
        xform_matrix = rot_matrix * inv_scale
        a0, a1 = xform_matrix[0]
        b0, b1 = xform_matrix[1]

        # At this point, the image will have been rotated around the top left corner,
        # rather than around the center of the image. 
        #
        # To fix this, we will see where the center of the image got sent by our transform,
        # and then undo that as part of the translation we apply.
        x_origin = float(width) / 2
        y_origin = float(width) / 2

        x_origin_shifted, y_origin_shifted = np.matmul(
            xform_matrix,
            np.array([x_origin, y_origin]),
        )

        x_origin_delta = x_origin - x_origin_shifted
        y_origin_delta = y_origin - y_origin_shifted

        # Combine our desired shifts with the rotation-induced undesirable shift
        a2 = x_origin_delta - (x_shift/(2*im_scale))
        b2 = y_origin_delta - (y_shift/(2*im_scale))

        # Return these values in the order that tf.contrib.image.transform expects
        return np.array([a0, a1, a2, b0, b1, b2, 0, 0]).astype(np.float32)

    def generate_random_transformation(self, scale_min=0.2, scale_max=0.6, width=psize, max_rotation=20):
        im_scale = np.random.uniform(low=scale_min, high=scale_max)

        padding_after_scaling = (1-im_scale) * width
        x_delta = np.random.uniform(-padding_after_scaling, padding_after_scaling)
        y_delta = np.random.uniform(-padding_after_scaling, padding_after_scaling)

        rot = np.random.uniform(-max_rotation, max_rotation)

        return self._transform_vector(width, 
                                      x_shift=x_delta,
                                      y_shift=y_delta,
                                      im_scale=im_scale, 
                                      rot_in_degrees=rot)    

model = ModelContainer()

# Create target labels

In [ ]:


def create_target_labels(scores, from_class, to_class):
    target_labels = np.zeros_like(scores)
    classes = np.argmax(scores[:, :, 1:], axis=2)+1

    for i, _ in enumerate(classes):
        for j, cls in enumerate(classes[i]):
            cls = to_class # Just perturb all of them!
            target_labels[i, j, cls] = 1

    return target_labels

# use half white images and half random noise images as the background images for the optimization
# print(model.batch_size_/2, type(model.batch_size_/2), type(psize)) # debug
white_imgs = np.ones((int(model.batch_size_ / 2), psize, psize, 3))
noisy_imgs = np.random.rand(int(model.batch_size_ / 2), psize, psize, 3) * 2 - 1.0
bg_imgs = np.concatenate([ noisy_imgs, white_imgs])

patch_transformations = np.zeros((model.batch_size_, 8))
for i in range(patch_transformations.shape[0]):
    patch_transformations[i, :] = model.generate_random_transformation()

_, scores, _ = model.inference(bg_imgs, patch_transformations)
target_labels = create_target_labels(scores, 13, TARGET_CLASS)

# Run the optimization

In [ ]:
# print(target_labels)

In [ ]:
model.reset_patch()

In [ ]:
import time
t0 = time.time()
for i in range(251): #251
    try:
        # Generate random transformations
        for j in range(patch_transformations.shape[0]):
            patch_transformations[j, :] = model.generate_random_transformation(scale_min=0.05, scale_max=0.3)

        # Update patch according to changed labels
        loss, second_stage_cls_loss, patch_loss = model.train_step(bg_imgs,
                                                                   patch_transformations,
                                                                   target_labels,
                                                                   learning_rate=1.0,
                                                                   patch_loss_weight=0.005)

        if (i % 10) == 0:
            t1 = time.time()
            print ('iter {} total loss: {} target loss: {} patch loss: {}'.format(i, loss, second_stage_cls_loss, patch_loss))
            print("time elapsed: ", t1-t0)
            t0 = t1
            model.inference(bg_imgs, patch_transformations)


    except KeyboardInterrupt:
        print ('iter {} total loss: {} target loss: {} patch loss: {}'.format(i, loss, second_stage_cls_loss, patch_loss))
        break

# Show the perturbation

In [ ]:
patch = model.get_patch()
showarray(PIL.Image.fromarray(patch))

# Test the perturbation

In [ ]:
patch = model.get_patch()
for j in range(patch_transformations.shape[0]):
    patch_transformations[j, :] = model.generate_random_transformation(scale_min=0.05, scale_max=0.3)
_ = model.inference(bg_imgs, patch_transformations)

# Save the perturbation

In [ ]:
PIL.Image.fromarray(patch).save('perturbation.png')